In [1]:
import pandas as pd 
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
csv_table = pd.read_table('../data/01_raw/SARI-Hospitalisierungsinzidenz.tsv' ,sep='\t')
csv_table.to_csv('../data/01_raw/SARI-Hospitalisierungsinzidenz.csv',index=False)


In [2]:
diagnosis_data = pd.read_csv('./CGM_data_01-2023_52-2023_extract.csv',sep=',')
prescription_data = pd.read_csv('./CGM_prescriptions_2023_01_2023_52.csv', sep=',')

In [5]:
def normalize_region(df): 
    """
    patient data / gp data uses seperated region NRW (Nordrhein and Westfalen-Lippe)
    This function merges them and stores all regions in region list
    """
    region = []
    
    for item in df['kvregion']:
        
        if item == 'Westfalen-Lippe' or item == 'Nordrhein':
            region.append('Nordrhein-Westfalen')
        else:
            region.append(item)
            
    return(region)

In [6]:
def week_to_monday(week_str):
    year, week = map(int, week_str.split('-'))
    # Calculate the date of the first day (Monday) of the given calendar week
    monday = datetime.datetime.strptime(f'{year}-W{week}-1', '%Y-W%W-%w').date()
    return monday.strftime('%Y-%m-%d')

In [7]:
regions = { "Deutschland": "DE",
            "Baden-Württemberg": "DE.BW",
            "Bayern": "DE.BY",
            "Berlin": "DE.BE",
            "Brandenburg": "DE.BB",
            "Bremen": "DE.HB",
            "Hamburg": "DE.HH",
            "Hessen": "DE.HE",
            "Mecklenburg-Vorpommern": "DE.MV",
            "Niedersachsen": "DE.NI",
            "Nordrhein-Westfalen": "DE.NW",
            "Rheinland-Pfalz": "DE.RP",
            "Saarland": "DE.SL",
            "Sachsen": "DE.SN",
            "Sachsen-Anhalt": "DE.ST",
            "Schleswig-Holstein": "DE.SH",
            "Thüringen": "DE.TH",
            }

In [8]:
def transforming_gp_data(data_frame, value_list):
    """
    Brings the gp data to the correct format by pivoting, and adding the correct date and region
    """
    data_frame_cp = data_frame
    data_frame_cp['region'] = normalize_region(data_frame_cp)
    pivot_df = data_frame_cp.pivot_table(index=['week', 'region'], columns=['category', 'age_group'], 
                                         values=value_list, aggfunc='sum')
    pivot_df.reset_index(inplace=True)
    pivot_df.columns = pivot_df.columns.map(' '.join)
    pivot_df.insert(0,"date",pivot_df.iloc[:,0].apply(lambda x: week_to_monday(x)))
    pivot_df.insert(1,"geography",pivot_df.iloc[:,2].apply(lambda x: regions[x]))
    pivot_df.fillna(0,inplace=True)
    pivot_df.columns = pivot_df.columns.str.strip()
    pivot_df=pivot_df.drop(columns=["week", "region"])
    return(pivot_df)

diagnosis_wide_df = transforming_gp_data(diagnosis_data,['distinct_patient_count','ratio','ratio_all'])
prescriptions_wide_df = transforming_gp_data(prescription_data, ['distinct_patient_count'])

In [9]:
diagnosis_wide_df = transforming_gp_data(diagnosis_data,['distinct_patient_count','ratio','ratio_all'])
prescriptions_wide_df = transforming_gp_data(prescription_data, ['distinct_patient_count'])

In [10]:
diagnosis_wide_df.to_csv("diagnoses.csv", index=False)
prescriptions_wide_df.to_csv("prescriptions.csv", index=False)